In [1]:
import os
import json
from glob import glob
os.chdir("..")

In [2]:
path = "/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/calculate_area individual files"
json_list = glob('/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/*.json')
json_list

['/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/get_weather_info.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/power.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/get_capital.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/calculate_trip_cost.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/count_letter.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/calculate_bmi.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/get_timezone.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/calculate_area.json']

In [3]:
queries = []
answers = []
for file_path in json_list[0:]:
    with open(file_path, 'r') as file:
        text = json.load(file)
        text = text["queries"]
        for item in text:
            queries.append(item["query"])
            answers.append(json.dumps(item["answer"]))
        

In [4]:
import pandas as pd

# check schema 
df = pd.DataFrame({"queries": queries, "answers": answers})
df["functions"] = df["answers"].apply(lambda x: json.loads(x).get("function", None))


print(f"there are {len(df)} rows at the beginning")

there are 5204 rows at the beginning


In [5]:
#filtered non function
df = df[~df["functions"].isna()]

function_names = df["functions"].unique()

sample_list = []
for func_name in function_names:
    sample_df  = df[df["functions"]== func_name].sample(n=210, random_state=1)
    sample_list.append(sample_df)
df = pd.concat(sample_list)

print(f"there are {len(df)} after filtering out non function and sampling")


there are 1680 after filtering out non function and sampling


In [6]:
# filter arguments
df["arguments"] = df["answers"].apply(lambda x: json.loads(x).get("arguments", None))
df = df[~df["arguments"].isna()]
df["arguments"] = df["arguments"].apply(json.dumps)

print(f"there are {len(df)} has arguments keys")


there are 1680 has arguments keys


In [7]:
from src.chi_functions import *
from src.edward_functions import *
from src.MT_functions import * 
from src.kevin_function import * 
import swifter

def call_function(info: pd.DataFrame):
    try:
        func_to_call = eval(info["functions"])
        
        arguments = json.loads(info["arguments"])
        return func_to_call(**arguments)
    except:
        return None
df["callable"] = df.swifter.apply(lambda x: call_function(x), axis=1)
df = df[~df["callable"].isna()]

len(df)



Sentiment Analysis Result: {"result": {"polarity": 1.0, "subjectivity": 0.75}}
Difference of Squares Result: {"result": 84}
Generated Password: {"result": "?+09{2>Wvg8sh/M#"}
Interest result:  {"result": 400.0}


/Users/cs/Library/Caches/pypoetry/virtualenvs/function-calling-project-_d8EX7TE-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Pandas Apply: 100%|██████████| 1680/1680 [07:40<00:00,  3.64it/s] 


1680

In [13]:
# save DataFrame
df.reset_index(drop=True, inplace=True)
df.to_csv("df_checkpoint_1.csv", index=None)

print(df["functions"].unique())
df[df["functions"] == "calculate_area"]


['get_weather_info' 'power' 'get_capital' 'calculate_trip_cost'
 'count_letter' 'calculate_bmi' 'get_timezone' 'calculate_area']


,queries,answers,functions,arguments,callable
1470,What is the area of a rectangle with length 2....,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""2.14"", ""width"": ""8.93"", ""round_res...","{""area"": ""19""}"
1471,What is the area of a square with length 7.654...,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""7.654"", ""width"": ""7.654"", ""round_r...","{""area"": ""59""}"
1472,What is the area of a rectangle with length 4....,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""4.567"", ""width"": ""7.890"", ""round_r...","{""area"": ""36""}"
1473,What is the area of a square with length 6.8 a...,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""6.8"", ""width"": ""1.5"", ""round_resul...","{""area"": ""10""}"
1474,What is the area of a square with length 8 and...,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""8"", ""width"": ""4.9"", ""round_result""...","{""area"": ""39""}"
...,...,...,...,...,...
1675,What is the area of a rectangle with length 7....,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""7.899"", ""width"": ""8"", ""round_resul...","{""area"": ""63""}"
1676,What is the area of a square with length 2.615...,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""2.615"", ""width"": ""8"", ""round_resul...","{""area"": ""21""}"
1677,Calculate the area of a square with length 8.9...,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""8.901"", ""width"": ""None"", ""round_re...","{""error"": ""Width must be a number.""}"
1678,What is the area of a rectangle with length 5....,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""5.005"", ""width"": ""8"", ""round_resul...","{""area"": ""40""}"


In [ ]:
import boto3
from langchain_aws import ChatBedrock
from botocore.config import Config
import time
from tqdm import tqdm
import random

def generate_response(df, llm):
    query_output = df["queries"]
    function_output = df["callable"]
    messages = [
    ("system", f"You are a helpful bot and you concisely answer user's query. Here is the information that can help you answer the question: {function_output}. Do not generate a response more than 25 words"),
    ("user", f"{query_output}")]
    response = llm.invoke(messages)

    return response.content

retry_config = Config(
    retries = {
        'max_attempts': 20,  # Customize the number of retry attempts
        'mode': 'adaptive'   # Or 'adaptive' for dynamically adjusting retries
    }
)

# Initialize the Bedrock client with retry configuration
bedrock_client = boto3.client('bedrock-runtime', config=retry_config)

# Use the ChatBedrock model in LangChain with the client that has retries
llm = ChatBedrock(
    client=bedrock_client,
    model_id="meta.llama3-1-8b-instruct-v1:0",  # Or "meta.llama3-1-70b-instruct-v1:0"
    temperature=0.2,
    max_tokens=None)



responses = []
numbers = []
# os.remove("responses.csv"
for item in tqdm(df.iterrows()):
    try:
        response = generate_response(item[1], llm)
        responses.append(response)
        numbers.append(item[0])
        time.sleep(random.random())
        if item[0] % 5 == 0 :
            temp_df = pd.DataFrame({"index": numbers, "responses": responses})
            temp_df.to_csv('responses.csv', mode='a', index=False, header=False)
            numbers = []
            responses = []
            

    except Exception as e:
        print(e)
        time.sleep(random.random()*70)
        response = generate_response(item[1], llm)
        responses.append(response)
        numbers.append(item[0])
        if item[0] % 5 ==0 :
            temp_df = pd.DataFrame({"index": numbers, "responses": responses})
            temp_df.to_csv('save_response.csv', mode='a', index=False, header=False)
            numbers = []
            responses = []
        print(item[1]["queries"])
        
        
    


75it [04:08,  3.65s/it]

In [12]:
response_df = pd.read_csv("../responses.csv", index_col=0, header=None)
response_df.rename(columns={1: "generated_response"}, inplace=True)
df = pd.read_csv("../df_checkpoint_1.csv", index_col=None)
df["generated_responses"] = temp_df["generated_response"]
df.to_csv("df_checkpoint_2.csv", index=None)


,queries,answers,functions,arguments,callable
273,What's the weather forecast for a snowy day in...,"{""function"": ""get_weather_info"", ""arguments"": ...",get_weather_info,"{""city"": ""Juneau""}","{""city"": ""Juneau"", ""time_in_PST"": ""2024-10-24 ..."
272,"Is it sunny today in PST in Bangor, Maine?","{""function"": ""get_weather_info"", ""arguments"": ...",get_weather_info,"{""city"": ""Bangor""}","{""city"": ""Bangor"", ""time_in_PST"": ""2024-10-24 ..."
329,Current time and weather info in Kuching,"{""function"": ""get_weather_info"", ""arguments"": ...",get_weather_info,"{""city"": ""Kuching""}","{""city"": ""Kuching"", ""time_in_PST"": ""2024-10-24..."
480,Can you give me an update on the weather in In...,"{""function"": ""get_weather_info"", ""arguments"": ...",get_weather_info,"{""city"": ""Indianapolis""}","{""city"": ""Indianapolis"", ""time_in_PST"": ""2024-..."
173,What is the time and temperature like right no...,"{""function"": ""get_weather_info"", ""arguments"": ...",get_weather_info,"{""city"": ""Funafuti""}","{""city"": ""Funafuti"", ""time_in_PST"": ""2024-10-2..."
...,...,...,...,...,...
4616,What is the area of a rectangle with length 7....,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""7.899"", ""width"": ""8"", ""round_resul...","{""area"": ""63""}"
4887,What is the area of a square with length 2.615...,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""2.615"", ""width"": ""8"", ""round_resul...","{""area"": ""21""}"
4621,Calculate the area of a square with length 8.9...,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""8.901"", ""width"": ""None"", ""round_re...","{""error"": ""Width must be a number.""}"
4960,What is the area of a rectangle with length 5....,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{""length"": ""5.005"", ""width"": ""8"", ""round_resul...","{""area"": ""40""}"
